In [ ]:
!pip install fiftyone

In [ ]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader


from src.datasets import MultimodalDataset

In [ ]:
dataset = load_from_hub("MatthiasCr/multimodal-shapes-subset", 
                         name="multimodal-shapes-subset",
                         # fewer workers and greater batch size to hopefully avoid getting rate limited
                         num_workers=2,
                         batch_size=200
                        )

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 32

label_map = {
    "cube": 0,
    "sphere": 1,
}

In [ ]:
img_transforms = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),  # Scales data into [0,1] TODO correct non deprecated version
])

In [ ]:
train_dataset = MultimodalDataset(dataset, "train", img_transforms, label_map)
val_dataset = MultimodalDataset(dataset, "val", img_transforms, label_map)

In [ ]:
loader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
)

rgb, lidar, label = next(iter(loader))

print(rgb.shape)
print(lidar.shape)
print(label.shape)